# HackZurich 2k21

In [1]:
# Filterning warnings from external libs

import warnings
warnings.filterwarnings('ignore')

In [2]:
# We would like to store our datasize on google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Copying dataset

!cp -r drive/My\ Drive/hakaton_ds/ds.zip ./

In [4]:
!nvidia-smi

Sun Sep 26 00:22:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!unzip -q ds.zip

In [6]:
import tensorflow as tf
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import os

In [7]:
# Creates spectogram from the mp3 file
def create_spectrogram(track_path):
  y, sr = librosa.load(track_path, duration=2.97)
  spect = librosa.feature.melspectrogram(y=y, sr=sr)
  return spect


In [8]:
# Preprocessing dataset

def create_ds(path):
  agros = os.listdir(os.path.join(path, 'agro'))
  calms = os.listdir(os.path.join(path, 'calm'))
  print(len(agros), len(calms))
  X = np.empty((0 , 128 , 128))
  Y = np.empty((0,))
  idx = 0
  ok_count = 0
  for agro_track, calm_track in zip(agros, calms):
    print(f'processing {idx} zip')
    try:
      spect_agro = create_spectrogram(os.path.join(path, 'agro', agro_track))
      X = np.append(X, [spect_agro], axis=0)
      Y = np.append(Y, [1])
      ok_count += 1
    except:
      print(f'agro {idx} corrupted, skipping')
    try:
      spect_calm = create_spectrogram(os.path.join(path, 'calm', calm_track))
      X = np.append(X, [spect_calm], axis=0)
      Y = np.append(Y, [0])
      ok_count += 1
    except:
      print(f'calm {idx} corrupted, skipping')
    idx+=1

  assert(len(X) == len(Y))
  perm = np.random.permutation(ok_count)
  X = X[perm]
  Y = Y[perm]

  X = np.array([x.reshape((128 , 128 , 1)) for x in X])
  Y = tf.keras.utils.to_categorical(Y, 2)
  valid_size = 120
  train_size = ok_count-valid_size
  x_train = X[:train_size] 
  y_train = Y[:train_size]
  x_valid = X[train_size:]
  y_valid = Y[train_size:]

  print(x_train[0])
  print(y_train[0])
  print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

  return x_train, y_train, x_valid, y_valid
  
x_train, y_train, x_valid, y_valid = create_ds('./')

720 720
processing 0 zip
processing 1 zip
processing 2 zip
processing 3 zip
processing 4 zip
processing 5 zip
processing 6 zip
processing 7 zip
processing 8 zip
processing 9 zip
processing 10 zip
processing 11 zip
processing 12 zip
processing 13 zip
processing 14 zip
processing 15 zip
processing 16 zip
processing 17 zip
processing 18 zip
processing 19 zip
processing 20 zip
processing 21 zip
processing 22 zip
processing 23 zip
processing 24 zip
processing 25 zip
processing 26 zip
processing 27 zip
processing 28 zip
processing 29 zip
processing 30 zip
processing 31 zip
processing 32 zip
processing 33 zip
processing 34 zip
processing 35 zip
processing 36 zip
processing 37 zip
processing 38 zip
processing 39 zip
processing 40 zip
processing 41 zip
processing 42 zip
processing 43 zip
processing 44 zip
processing 45 zip
processing 46 zip
processing 47 zip
processing 48 zip
processing 49 zip
processing 50 zip
processing 51 zip
processing 52 zip
processing 53 zip
processing 54 zip
processing 5

In [9]:
# Store .npy files for later usage
np.save('x_train.npy' , x_train)
np.save('y_train.npy' , y_train)
np.save('x_valid.npy' , x_valid)
np.save('y_valid.npy' , y_valid)

!cp x_train.npy y_train.npy x_valid.npy y_valid.npy drive/My\ Drive/

In [10]:
model = tf.keras.Sequential()
input_shape = (128 , 128 , 1)
model.add(tf.keras.layers.Conv2D(64 , (5 , 5) , strides=(1 , 1), input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((4,2) , strides=(4 , 2)))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Conv2D(128 , (5 , 5) , padding='valid'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((4 , 2) , strides=(4 , 2)))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Conv2D(128 , (5 , 5) , padding='valid'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Activation('softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 64)      1664      
_________________________________________________________________
batch_normalization (BatchNo (None, 124, 124, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 62, 64)        0         
_________________________________________________________________
activation (Activation)      (None, 31, 62, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 58, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 58, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 29, 128)        0

In [12]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
]

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    x=x_train,
    y=y_train,
    epochs=50,
    batch_size=64,
    validation_data=(x_valid , y_valid),
    callbacks=callbacks,
)

Epoch 1/50
21/21 [==============================] - 36s 140ms/step - loss: 0.7035 - accuracy: 0.6875 - val_loss: 4.2824 - val_accuracy: 0.5583
Epoch 2/50
21/21 [==============================] - 2s 92ms/step - loss: 0.5568 - accuracy: 0.7224 - val_loss: 1.3920 - val_accuracy: 0.6833
Epoch 3/50
21/21 [==============================] - 2s 92ms/step - loss: 0.5470 - accuracy: 0.7529 - val_loss: 1.0797 - val_accuracy: 0.7417
Epoch 4/50
21/21 [==============================] - 2s 91ms/step - loss: 0.4382 - accuracy: 0.7932 - val_loss: 0.7134 - val_accuracy: 0.7500
Epoch 5/50
21/21 [==============================] - 2s 91ms/step - loss: 0.3632 - accuracy: 0.8456 - val_loss: 3.7605 - val_accuracy: 0.5250
Epoch 6/50
21/21 [==============================] - 2s 92ms/step - loss: 0.3300 - accuracy: 0.8631 - val_loss: 0.9785 - val_accuracy: 0.7750
Epoch 7/50
21/21 [==============================] - 2s 92ms/step - loss: 0.2341 - accuracy: 0.9110 - val_loss: 1.3624 - val_accuracy: 0.7750
Epoch 8/50


In [13]:
# Save checkpoint
model.save('hackzurich.savedmodel')

INFO:tensorflow:Assets written to: hackzurich.savedmodel/assets


In [16]:
# Copying to the disk
!cp -r hackzurich.savedmodel/ drive/My\ Drive

In [17]:
def prepare_infer_from_sample(path: str):
  X = np.empty((0 , 128 , 128))
  spect = create_spectrogram(path)
  X = np.append(X, [spect], axis=0)
  X = np.array([x.reshape((128 , 128 , 1)) for x in X])
  return X

In [18]:
# Final evaluation
model.evaluate(x_valid, y_valid)

4/4 [==============================] - 0s 70ms/step - loss: 0.4900 - accuracy: 0.8250


[0.4899863302707672, 0.824999988079071]